In [0]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo

In [0]:
__all__ = ['SqueezeNet', 'squeezenet1_0', 'squeezenet1_1']


In [0]:
model_urls = {
    'squeezenet1_0': 'https://download.pytorch.org/models/squeezenet1_0-a815701f.pth',
    'squeezenet1_1': 'https://download.pytorch.org/models/squeezenet1_1-f364aa15.pth',
}

In [0]:
class Fire(nn.Module):

    def __init__(self, inplanes, squeeze_planes,
                 expand1x1_planes, expand3x3_planes):
        super(Fire, self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes,
                                   kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes,
                                   kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat([
            self.expand1x1_activation(self.expand1x1(x)),
            self.expand3x3_activation(self.expand3x3(x))
        ], 1)


In [0]:


class SqueezeNet(nn.Module):

    def __init__(self, version=1.0, num_classes=1000):
        super(SqueezeNet, self).__init__()
        if version not in [1.0, 1.1]:
            raise ValueError("Unsupported SqueezeNet version {version}:"
                             "1.0 or 1.1 expected".format(version=version))
        self.num_classes = num_classes
        if version == 1.0:
            self.features = nn.Sequential(
                nn.Conv2d(3, 96,kernel_size=7, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(96, 16, 64, 64),
                #Fire(128, 16, 64, 64),
                Fire(128, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                #Fire(256, 32, 128, 128),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(512, 64, 256, 256),
            )
        else:
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(64, 16, 64, 64),
                Fire(128, 16, 64, 64),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(128, 32, 128, 128),
                Fire(256, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                Fire(512, 64, 256, 256),
            )
        # Final convolution is initialized differently form the rest
        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            final_conv,
            nn.ReLU(inplace=True),
            nn.AvgPool2d(13, stride=1)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    init.normal(m.weight.data, mean=0.0, std=0.01)
                else:
                    init.kaiming_uniform(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x.view(x.size(0), self.num_classes)

In [0]:
def squeezenet1_0(pretrained=False, **kwargs):
    r"""SqueezeNet model architecture from the `"SqueezeNet: AlexNet-level
    accuracy with 50x fewer parameters and <0.5MB model size"
    <https://arxiv.org/abs/1602.07360>`_ paper.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = SqueezeNet(version=1.0, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['squeezenet1_0']))
    return model

In [7]:
seq=squeezenet1_0(pretrained=False,num_classes=10).cuda()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.


In [8]:
# total parameters
model=seq
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

678682

In [0]:
#CIFAR10 squeezenet 
resize=227

import torchvision.datasets as datasets
import torchvision.transforms as transforms


#model=alexnet(pretrained=False,num_classes=10).cuda()
model=squeezenet1_0(pretrained=False,num_classes=10).cuda()




#dataiter = iter(testloader)
#images, labels = dataiter.next()
#outputs = model(images.cuda())
#outputs = model(images)

def evaluate(model):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          outputs = model(images.cuda())
          #outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels.cuda()).sum().item()
          #correct += (predicted == labels).sum().item()
  print('Accuracy of the network on the 10000 test images: %d %%' % (
      100 * correct / total))

  acc= 100 * correct / total







import torch
import torchvision
import torchvision.transforms as transforms

#transform1
transform = transforms.Compose(
    [#transforms.Resize((32,32)), #kept to orignal size 
     
     transforms.ToTensor(),
     
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
     ])


#transform2 (transform_train)



transform = transforms.Compose([
    transforms.Resize((resize,resize)),
    transforms.RandomCrop(resize, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])



transformtest=transforms.Compose([
    transforms.Resize((resize,resize)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)



trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transformtest)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
import matplotlib.pyplot as plt
import numpy as np


import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#criterion = nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9,weight_decay=5e-4)
optimizer = optim.Adam(model.parameters())
n=0
optimizer.zero_grad()


for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    print(epoch)
    evaluate(model)
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.cuda())
        loss = criterion(outputs.cuda(), labels.cuda())
        loss.backward()
        optimizer.step()
        #print(i)
        
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
    

print('Finished Training') 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.


Files already downloaded and verified
Files already downloaded and verified
0
Accuracy of the network on the 10000 test images: 11 %
1
Accuracy of the network on the 10000 test images: 37 %
2
Accuracy of the network on the 10000 test images: 44 %
3
Accuracy of the network on the 10000 test images: 50 %
4
Accuracy of the network on the 10000 test images: 53 %
5
Accuracy of the network on the 10000 test images: 55 %
6
Accuracy of the network on the 10000 test images: 59 %
7
Accuracy of the network on the 10000 test images: 66 %
8
Accuracy of the network on the 10000 test images: 69 %
9
Accuracy of the network on the 10000 test images: 70 %
10
Accuracy of the network on the 10000 test images: 73 %
11
Accuracy of the network on the 10000 test images: 73 %
12
Accuracy of the network on the 10000 test images: 76 %
13
Accuracy of the network on the 10000 test images: 76 %
14
Accuracy of the network on the 10000 test images: 76 %
15
Accuracy of the network on the 10000 test images: 76 %
16
Acc